# DYNAMIC ROUTING SIMULATOR

This takes you step by step for creating a dynamic routing simulator which leverages the Carto Overture Maps, Open Route Service as well as many of the advanced analytical functions within Snowflake.  Let's get started.

## Initial Setup

### Open Route Service API
- Before you start, sign up for a free open route service account.  Open Route service gives you limited free calls to each service.  If you would like these limitations removed, a local installation is available - which can be hosted in Snowpark Container Services.  For this demonstration, we will leverage the service which is hosted within openrouteservice.org
https://openrouteservice.org/
- once you have created an open route service account and signed in, create a token.

### Marketplace data for simulation generation
- go to the market place and search for **Carto Overture Maps**
- Install the Places dataset and keep the name as it is
- Grant permissions for the public role

In [ ]:
GRANT IMPORTED PRIVILEGES ON DATABASE "OVERTURE_MAPS__PLACES" TO ROLE PUBLIC

### CREATE A SECRET
The secret provided by open route service requires save storage.  Use the newly created api token for the **SECRET_STRING**. 

In [ ]:
CREATE SECRET IF NOT EXISTS UTILS.ROUTING_TOKEN
  TYPE = GENERIC_STRING
  SECRET_STRING = '<replace with your secret token>'
  COMMENT = 'token for routing demo' 

#### CREATE A NETWORK RULE AND EXTERNAL INTEGRATION
Before we call the API, we need to allow external activity from Snowflake to the route service URL.  This is achieved simply through a network rule.  You then create an external access integration - this allows the user to call the api using the predefined secret and only via the allowed network rule.

Next, we need to attach this rule to a new external access integration which in this case, we are calling open_route_api. All of the functions that we will create will refer to this integration.

In [ ]:
CREATE OR REPLACE NETWORK RULE open_route_api
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('api.openrouteservice.org');


CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION open_route_integration
  ALLOWED_NETWORK_RULES = (open_route_api)
  ALLOWED_AUTHENTICATION_SECRETS = all
  ENABLED = true;

### Creating API functions which we will use in our app.
Now we have created the secure integration to the service, we will create functions which we will call in the app.  These functions will call the relevant api within the service needed to perform each task.

#### 1.  Simple Directions - allows simple directions from one point to another.
You can load multiple start/end points to get directions for multiple places). You also need to specify the method such as driving-car.  This function is leveraging a GET request from the api.  You will note that the function is written in python - the service itself has handy python snippets to help curate the function.  
click [here](https://openrouteservice.org/dev/#/api-docs/v2/directions/{profile}/post) to navigate to the directions service playground

In [ ]:
CREATE OR REPLACE FUNCTION UTILS.DIRECTIONS (method varchar, jstart array, jend array)
RETURNS VARIANT
language python
runtime_version = 3.9
handler = 'get_directions'
external_access_integrations = (OPEN_ROUTE_INTEGRATION)
PACKAGES = ('snowflake-snowpark-python','requests')
SECRETS = ('cred' = UTILS.ROUTING_TOKEN )

AS
$$
import requests
import _snowflake
def get_directions(method,jstart,jend):
    request = f'''https://api.openrouteservice.org/v2/directions/{method}'''
    key = _snowflake.get_generic_secret_string('cred')

    PARAMS = {'api_key':key,
            'start':f'{jstart[0]},{jstart[1]}', 'end':f'{jend[0]},{jend[1]}'}

    r = requests.get(url = request, params = PARAMS)
    response = r.json()
    
    return response
$$;

Here is how to call the function.  This will return a json object.  You will notice that I have already navigated to the features[0]:geometry part of the object.  The reason why features nested in an array is because you can call the api to retrieve multiple directions.  Standard semi structured support allows me to extract key pieces of informaton out.

In [ ]:

select  GEO:features[0]:geometry GEO, 
GEO:features[0]:properties:segments INSTRUCTIONS, 
DIV0(GEO:features[0]:properties:summary:distance::FLOAT,1000) DISTANCE_KM,
DIV0(GEO:features[0]:properties:summary:duration::FLOAT,60) DURATION_MINS

from

(select UTILS.directions('driving-car',[-2.263796,52.344367],[0.0813,51.5186]) GEO)


I will now introduce you to the **pydeck** library.  This library allows you to visualise multiple map layers which can include polygons,linestrings and points.  Below is rendering a simple line string which gives me the path to drive to the Snowflake Office from my house

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
import pandas as pd
import pydeck as pdk
import json
session = get_active_session()


data = directions_try_out.to_df().select('GEO').to_pandas()
data["coordinates"] = data["GEO"].apply(lambda row: json.loads(row)["coordinates"])

vehicle_1_path = pdk.Layer(
type="PathLayer",
data=data,
pickable=True,
get_color=[253, 180, 107],
width_scale=20,
width_min_pixels=4,
width_max_pixels=7,
get_path="coordinates",
get_width=5)

view_state = pdk.ViewState(latitude=52.344367, longitude=-2.263796, zoom=7)
st.pydeck_chart(pdk.Deck(layers=[vehicle_1_path],map_style=None,initial_view_state=view_state,height=900))

#### 2. Advanced directions - create routes which include multiple way points. 
Although the function above is nice and simple for directions that stops at only one place, vehicle route plans that contain multiple drop offs will need to effectively 'detour' between the starting point and destination.  This is where we utilise **way points**.

Each stop is loaded into one array.  You also need to specify the method such as driving-car.  This type of api uses a PUT request - you will see how this is articulated in the code below.

In [ ]:
CREATE OR REPLACE FUNCTION UTILS.DIRECTIONS_with_way_points (method varchar, locations array)
RETURNS VARIANT
language python
runtime_version = 3.9
handler = 'get_directions'
external_access_integrations = (OPEN_ROUTE_INTEGRATION)
PACKAGES = ('snowflake-snowpark-python','requests')
SECRETS = ('cred' = UTILS.ROUTING_TOKEN )

AS
$$
import requests
import _snowflake

def get_directions(method,locations):
    request_directions = f'''https://api.openrouteservice.org/v2/directions/{method}/geojson'''
    key = _snowflake.get_generic_secret_string('cred')

    HEADERS = { 'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
               'Authorization':key,
               'Content-Type': 'application/json; charset=utf-8'}

    body = {"coordinates":locations}

    r = requests.post(url = request_directions,json = body, headers=HEADERS)
    response = r.json()
    
    return response
$$;

Lets now take you through calling the service which pushes in multiple way points.  Firstly, you will create a series of waypoints in an array.  Python is a great way to inject array variables into our sql function via a notebook for testing purposes.

In [ ]:
way_points = [
  [
    -97.728009,
    30.263267
  ],
  [
    -97.7168732,
    30.3508968
  ],
  [
    -97.6812713,
    30.3052591
  ],
  [
    -97.7218991,
    30.3007635
  ],
  [
    -97.82502,
    30.08664
  ],
  [
    -97.728009,
    30.263267
  ]
]

We will then use Snowpark to persist the waypoints into a dataframe.  This will be used to render the points on a map. 

In [ ]:
waypoints = session.create_dataframe(way_points)\
.with_column_renamed(col('_1'),'LON').with_column_renamed(col('_2'),'LAT')
waypoints

Now lets call the waypoint function which will return a line string containing the directions.  We do have the capability to extract the instructions for each waypoint - all this is stored in the same way as the simple directions service.  However, for this exercise we are interested in the line string only.  This will in turn form the second layer of the map.

In [ ]:
SELECT UTILS.DIRECTIONS_with_way_points('driving-car',{{way_points}}):features[0]:geometry GEO

We will now visualise the the directions with way points again using pydeck.  This incorporates two layers - one for the line string and the second for each way point 

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
import pandas as pd
import pydeck as pdk
import json
session = get_active_session()

drop_offs = waypoints.to_pandas()
data = try_way_points.to_df().select('GEO').to_pandas()
data["coordinates"] = data["GEO"].apply(lambda row: json.loads(row)["coordinates"])

scatter = pdk.Layer(
            'ScatterplotLayer',
            data=drop_offs,
            get_position='[LON, LAT]',
            get_color='[0,187,2]',
            get_radius=600,
            pickable=True)


vehicle_1_path = pdk.Layer(
type="PathLayer",
data=data,
pickable=True,
get_color=[253, 180, 107],
width_scale=20,
width_min_pixels=4,
width_max_pixels=7,
get_path="coordinates",
get_width=6)

view_state = pdk.ViewState(latitude=30.08664, longitude=-97.82502, zoom=10)
st.pydeck_chart(pdk.Deck(layers=[vehicle_1_path,scatter],map_style=None,initial_view_state=view_state,height=900))

#### 3. Route Optimisation Function - used to find the best optimal workload for each vehicle based on all the job requirements and the vehicles that are available.  

The job detailed requirements such as skills, capacity requirements and delivery slots are sent in the first array. Any available vehicle specifications are sent in the second array.

In [ ]:
CREATE OR REPLACE FUNCTION UTILS.OPTIMIZATION (jobs array, vehicles array)
RETURNS VARIANT
language python
runtime_version = 3.9
handler = 'get_optimization'
external_access_integrations = (OPEN_ROUTE_INTEGRATION)
PACKAGES = ('snowflake-snowpark-python','requests')
SECRETS = ('cred' = UTILS.ROUTING_TOKEN )

AS
$$
import requests
import _snowflake
def get_optimization(jobs,vehicles):
    request_optimization = f'''https://api.openrouteservice.org/optimization'''
    key = _snowflake.get_generic_secret_string('cred')
    HEADERS = { 'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
               'Authorization':key,
               'Content-Type': 'application/json; charset=utf-8'}

    body = {"jobs":jobs,"vehicles":vehicles}

    r = requests.post(url = request_optimization,json = body, headers=HEADERS)
    response = r.json()
    
    return response
$$;

Now let's test this function out.  Below are two variables created for jobs and vehicles.  The optimisation service has documentation regarding the sorts of information you can pass through for both jobs and vehicles.  I am passing information such as a skill, the drop off location, capacity requirements and a skill.  The **skill** number needs to match to assign a job to a vehicle.  If the skill number on the job does not match any of the vehicles, the job will be flagged as **unassigned**.  This number could effectively relate to special requirement such as the vehicle must contain a fridge.

In [ ]:
jobs = [
  {
    "capacity": [
      2
    ],
    "id": 1,
    "location": [
      -4.613748000000000e-01,
      5.152569760000000e+01
    ],
    "skills": [
      1
    ],
    "time_window": [
      32400,
      36000
    ]
  },
  {
    "capacity": [
      2
    ],
    "id": 6,
    "location": [
      -3.355460000000000e-01,
      5.150566550000000e+01
    ],
    "skills": [
      1
    ],
    "time_window": [
      36000,
      54000
    ]
  }

]

vehicles = [
  {
    "capacity": [
      2
    ],
    "end": [
      -4.467074000000000e-01,
      5.146048020000000e+01
    ],
    "id": 1,
    "profile": "driving-car",
    "skills": [
      1
    ],
    "start": [
      -4.467074000000000e-01,
      5.146048020000000e+01
    ],
    "time_windows": [
      28800,
      61200
    ]
  },
]

**Now** we will test the function out - we will apply the two variant variables to the function and a json object will be returned.

In [ ]:
SELECT UTILS.OPTIMIZATION({{jobs}},{{vehicles}}) OPTIMIZATION

Let's now have a look at the json object. You will see that the array object is broken down into routes.  Each route will be assigned a vehicle and that vehicle will be allocated delivery locations and times.  There will also be infomation about the duration and may include details such as waiting times.

In [ ]:
test_route_optimisation.to_df().collect()[0][0]

We will use standard Geospatial functions to collect all the points into one row.  This will be used to create a line in pydeck.  In addition, I have used ST_ENVELOPE to create a bounding box for the collection of points.  This is so I can work out the overall centre point of each delivery location.  This will be useful for centering the rendered pydeck map.

In [ ]:
from snowflake.snowpark.functions import *
routes = test_route_optimisation.to_df().select(col('OPTIMIZATION')['routes'][0]['steps'].alias('steps'))

routes = routes.join_table_function('flatten',col('steps')).select('VALUE')
routes = routes.select(col('VALUE')['arrival'].alias('ARRIVAL'),
              col('VALUE')['arrival'].alias('duration'),
              col('VALUE')['load'].alias('load'),
              col('VALUE')['id'].alias('Job ID'),
              col('VALUE')['location'].alias('Location'),
              col('VALUE')['service'].alias('Service'))
            
line = routes.select(call_function('ST_COLLECT',call_function('st_makepoint',col('Location')[0],col('Location')[1])).alias('LINE')) 
line = line.with_column('CENTROID',call_function('ST_CENTROID',(call_function('ST_ENVELOPE',col('LINE')))))
line

We will view the journey by combining all drop offs.  Of course the journey will involve roads, that is where the waypoint function will be used - the returned results of each drop off will be pushed into the waypoint function.  The pydeck visualisation compares straight line with directions as two distinct layers

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
import pandas as pd
import pydeck as pdk
import json
session = get_active_session()

center = line.select(call_function('ST_X',col('CENTROID')).alias('X'),
                     call_function('ST_Y',col('CENTROID')).alias('Y')).limit(1).to_pandas()

LAT = center.Y.iloc[0]
LON = center.X.iloc[0]

directions = line.select(call_function('st_asgeojson',col('line'))['coordinates'].alias('line'))
directions = directions.with_column('line',
                                    call_function('UTILS.DIRECTIONS_with_way_points',
                                                  'driving-car',
                                                  col('line'))['features'][0]['geometry'])

st.write(directions)
data = directions.select('LINE').to_pandas()
data["coordinates"] = data["LINE"].apply(lambda row: json.loads(row)["coordinates"])
data2 = line.to_pandas()
data2["coordinates"] = data2["LINE"].apply(lambda row: json.loads(row)["coordinates"])

vehicle_1_path = pdk.Layer(
type="PathLayer",
data=data,
pickable=True,
get_color=[253, 180, 107],
width_scale=20,
width_min_pixels=4,
width_max_pixels=7,
get_path="coordinates",
get_width=5)

vehicle_1_straight_line = pdk.Layer(
type="PathLayer",
data=data2,
pickable=True,
get_color=[255,0, 107],
width_scale=20,
width_min_pixels=4,
width_max_pixels=7,
get_path="coordinates",
get_width=5)

view_state = pdk.ViewState(latitude=LAT, longitude=LON, zoom=10)
st.markdown('Waypoints joined together as a straight line')
st.pydeck_chart(pdk.Deck(layers=[vehicle_1_path,vehicle_1_straight_line],map_style=None,initial_view_state=view_state,height=900))

#### 4. Isochrone Function.  
This is to create a polygon which creates a catchment boundary based on how long it would take to get there.  Orders may only be accepted if you can fullfill them based on how quickly drivers can get anywhere within a given timeframe.  Isochrones could be utilised for other usecases outside of route optimisation such as market analysis in a given catchment.

In [ ]:
CREATE OR REPLACE FUNCTION UTILS.ISOCHRONES(method string, lon float, lat float, range int)
RETURNS VARIANT
language python
runtime_version = 3.9
handler = 'get_isochrone'
external_access_integrations = (OPEN_ROUTE_INTEGRATION)
PACKAGES = ('snowflake-snowpark-python','requests')
SECRETS = ('cred' = UTILS.ROUTING_TOKEN )

AS
$$
import requests
import _snowflake
def get_isochrone(method,lon,lat,range):
    request_isochrone = f'''https://api.openrouteservice.org/v2/isochrones/{method}'''
    key = _snowflake.get_generic_secret_string('cred')
    HEADERS = { 'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
               'Authorization':key,
               'Content-Type': 'application/json; charset=utf-8'}

    body = {'locations':[[lon,lat]],
                    'range':[range*60],
                    'location_type':'start',
                    'range_type':'time',
                    'smoothing':10}

    r = requests.post(url = request_isochrone,json = body, headers=HEADERS)
    response = r.json()
    
    return response
$$;

The function is then tested by focussing on a specific point.  It will generate a polygon which represents everywhere a driver can get to within a 20 minute timeframe.

In [ ]:
select UTILS.ISOCHRONES('cycling-regular', 2.351,48.85, 20):features[0]:geometry GEO, ST_CENTROID(TO_GEOGRAPHY(GEO)) CENTROID, ST_X(CENTROID) LON, ST_Y(CENTROID) LAT

This is an example of what an isochrone looks like in pydeck.

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
import pandas as pd
import pydeck as pdk
import json
session = get_active_session()


data = isochrones_try.to_df().select('GEO','LON','LAT').to_pandas()
LAT = data.LAT.iloc[0]
LON = data.LON.iloc[0]
data["coordinates"] = data["GEO"].apply(lambda row: json.loads(row)["coordinates"])

isochrone = pdk.Layer(
"PolygonLayer",
data,
opacity=0.7,
get_polygon="coordinates",
filled=True,
get_line_color=[41,181,232],
get_fill_color=[200,230,242],
get_line_width=10,
line_width_min_pixels=6,
auto_highlight=True,
pickable=False)

view_state = pdk.ViewState(latitude=LAT, longitude=LON, zoom=10)
st.pydeck_chart(pdk.Deck(layers=[isochrone],map_style=None,initial_view_state=view_state,height=900))

## Create Base Data Table for App
Now we will focus on putting components together for a streamlit application.  We are aiming to create a dynamic routing analyser.  For this, we will need to generate jobs, vehicles and depots based on any location in the world.  Carto overture maps will help as it contains points of interests anywhere in the world. Cortex complete is also used for a simple 'geocoding' mechanism where a user will type in a key word and cortex will generate the coordinates that matches what central location the user wants to base the routing against

#### 1. Job Template
This template will be used to simulate jobs.  You can change the base values in this notebook - which will effect what sample jobs will be generated.  This template generates a sample of 29 jobs.  The location of these jobs and industry will be dynamic and depends on user interaction with the app.

In [ ]:
CREATE OR REPLACE TABLE JOB_TEMPLATE (

ID INT AUTOINCREMENT PRIMARY KEY,
slot_start INT NOT NULL,
slot_end INT,
skills INT,
product STRING,
status STRING DEFAULT 'active'

)



In [ ]:
INSERT INTO JOB_TEMPLATE (slot_start, slot_end, skills, product, status) VALUES
(9, 10, 1, 'pa', 'active'),
(11, 15, 2, 'pb', 'active'),
(16, 18, 2, 'pb', 'active'),
(11, 13, 3, 'pc', 'active'),
(7, 16, 3, 'pc', 'active'),
(10, 15, 2, 'pa', 'active'),
(10, 15, 2, 'pa', 'active'),
(7, 16, 1, 'pa', 'active'),
(9, 18, 2, 'pb', 'active'),
(13, 18, 2, 'pb', 'active'),
(13, 18, 2, 'pb', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 3, 'pc', 'active'),
(11, 15, 2, 'pb', 'active'),
(16, 18, 2, 'pb', 'active'),
(11, 13, 1, 'pa', 'active'),
(7, 16, 1, 'pa', 'active'),
(10, 15, 2, 'pb', 'active'),
(10, 15, 2, 'pb', 'active'),
(7, 16, 1, 'pa', 'active'),
(9, 18, 2, 'pb', 'active'),
(13, 18, 2, 'pb', 'active'),
(13, 18, 2, 'pb', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 3, 'pc', 'active');


In [ ]:
SELECT * FROM JOB_TEMPLATE

#### 2. Create a base table with search optimisation to filter quickly on categories and geographies

Here we are using the places dataset which is provided by carto overture maps.  We are copying the data to add search optimisation on the categories of place as well as the geography.  We will be leveraging the 'Search' function in the app to search through columns that may be relevant to the selected industry.

In [ ]:
CREATE OR REPLACE TABLE places as 

select 

any_value(GEOMETRY) GEOMETRY,

PHONES:list[0]['element']::text PHONES,
CATEGORIES:primary::text CATEGORY,
NAMES:primary::text NAME,
ADDRESSES:list[0]['element'] ADDRESS,

array_agg(value:element) as ALTERNATE 

from

(SELECT PHONES,CATEGORIES,NAMES,ADDRESSES,GEOMETRY,

categories:alternate:list AS LIST

from OVERTURE_MAPS__PLACES.CARTO.PLACE), 

--where CATEGORIES:primary is not null),
LATERAL FLATTEN(LIST) GROUP BY ALL;

ALTER TABLE places ADD SEARCH OPTIMIZATION ON EQUALITY(ALTERNATE);
ALTER TABLE places ADD SEARCH OPTIMIZATION ON GEO(GEOMETRY);

SELECT * FROM PLACES limit 3

In [ ]:
select * EXCLUDE (GEOMETRY,GEO),ST_X(GEOMETRY) LON,ST_Y(GEOMETRY) LAT from PLACES A INNER JOIN
(select GEO from {{isochrones_try}}) B ON

ST_WITHIN(TO_GEOGRAPHY(A.GEOMETRY),TO_GEOGRAPHY(B.GEO))

where CATEGORY = 'hotel'

Here, we are using the Scatter Plot layer to only view places by the isochrone 

In [ ]:
center = isochrones_try.to_df().limit(1).to_pandas()

BLAT = center.LAT.iloc[0]
BLON = center.LON.iloc[0]

context = pdk.Layer(
'ScatterplotLayer',
places_on_isochrone.to_pandas(),
get_position=['LON', 'LAT'],
filled=True,
stroked=False,
radius_min_pixels=6,
radius_max_pixels=20,
auto_highlight=True,
get_fill_color=[41, 181, 232],
pickable=True)
st.divider()

view_state = pdk.ViewState(latitude=BLAT, longitude=BLON, zoom=10)
st.pydeck_chart(pdk.Deck(layers=[context],map_style=None,initial_view_state=view_state,height=900))

#### Below are all the categories you could use to simulate distribution points
The app will search through names, categories and alternative categories in order to retrieve the relevant data

In [ ]:
SELECT DISTINCT CATEGORY FROM (
select DISTINCT VALUE::TEXT CATEGORY FROM PLACES, LATERAL FLATTEN (ALTERNATE)

UNION 

SELECT DISTINCT CATEGORY FROM PLACES) WHERE SEARCH((CATEGORY),'food')

### 4. Creating an Industry Lookup

Depending on the selected industry, the app will 'lookup' specifics in order to retrieve/ filter the right information.  It is also used to generate relevant product categories. In this notebook, more industries can be added.

In [ ]:
CREATE OR REPLACE TABLE LOOKUP (
    INDUSTRY STRING,
    PA STRING,
    PB STRING,
    PC STRING,
    IND ARRAY,
    IND2 ARRAY,
    CTYPE ARRAY,
    STYPE ARRAY
);

INSERT INTO LOOKUP (INDUSTRY, PA, PB, PC, IND,IND2, CTYPE, STYPE) 
SELECT
    'healthcare', 
    'flammable', 
    'sharps', 
    'temperature-controlled', 
    ARRAY_CONSTRUCT('hospital health pharmaceutical drug healthcare pharmacy surgical'), 
    ARRAY_CONSTRUCT('supplies warehouse depot distribution wholesaler distributors'), 
    ARRAY_CONSTRUCT('hospital', 'family_practice', 'dentist','pharmacy'), 
    ARRAY_CONSTRUCT('Can handle potentially explosive goods', 'Can handle instruments that could be used as weapons', 'Has a fridge')
UNION ALL
SELECT
    'Food', 
    'Fresh Food Order', 
    'Frozen Food Order', 
    'Non Perishable Food Order', 
    ARRAY_CONSTRUCT('food vegatables meat vegatable'),
    ARRAY_CONSTRUCT('wholesaler warehouse factory processing distribution distributors'), 
    ARRAY_CONSTRUCT('supermarket', 'restaurant', 'butcher_shop'), 
    ARRAY_CONSTRUCT('Can deliver Fresh Food', 'Has a Fridge', 'Premium Delivery')
UNION ALL
SELECT
    'Cosmetics', 
    'Hair Products', 
    'Electronic Goods', 
    'Make-up', 
    ARRAY_CONSTRUCT('hair cosmetics make-up beauty'),
    ARRAY_CONSTRUCT('wholesaler warehouse factory supplies distribution distributors'), 
    ARRAY_CONSTRUCT('supermarket', 'outlet', 'fashion'), 
    ARRAY_CONSTRUCT('Can deliver Fresh Food', 'Has a Fridge', 'Premium Delivery')

    ;



In [ ]:
SELECT * FROM LOOKUP

#### Here is an example of how the lookup workes for healthcare for flammable products

In [ ]:
select * from JOB_TEMPLATE a 

inner join

(select PA from lookup where industry = 'healthcare') b

on PRODUCT = 'pa'

## Streamlit App

Now you have setup the functions and standing data tables, you can now have a go at running the streamlit app.  The streamlit application is an example of what a basic route optimisation simulator could look like.  Feel free to make any changes as you see fit.

- Within Projects / Streamlit you should see a new application
- Click on the app to run the route optimisation simulator

In [ ]:
GRANT SELECT ON ALL TABLES IN SCHEMA ROUTING TO ROLE PUBLIC;

GRANT USAGE ON ALL FUNCTIONS IN SCHEMA UTILS TO ROLE PUBLIC;